## Exploratory Data Analysis and Visualization
Here, we check our data to make sure all NaNs correspond to approved claims, and not rejected claims. First, we import packages and read our csv.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
cmm = pd.read_csv("Data/CMM.csv")

In [3]:
#What does it look like?
cmm.head()

,dim_claim_id,dim_pa_id,dim_date_id,bin,drug,reject_code,pharmacy_claim_approved,date_val,calendar_year,calendar_month,calendar_day,day_of_week,is_weekday,is_workday,is_holiday,correct_diagnosis,tried_and_failed,contraindication,pa_approved
0,1,1.0,1,417380,A,75.0,0,2017-01-01,2017,1,1,1,0,0,1,1.0,1.0,0.0,1.0
1,2,NaN,1,999001,A,NaN,1,2017-01-01,2017,1,1,1,0,0,1,NaN,NaN,NaN,NaN
2,3,2.0,1,417740,A,76.0,0,2017-01-01,2017,1,1,1,0,0,1,1.0,0.0,0.0,1.0
3,4,NaN,1,999001,A,NaN,1,2017-01-01,2017,1,1,1,0,0,1,NaN,NaN,NaN,NaN
4,5,NaN,1,417740,A,NaN,1,2017-01-01,2017,1,1,1,0,0,1,NaN,NaN,NaN,NaN


First, we make sure claims aren't duplicated in this dataset. To do this, we compare the number of entries to the number of claim ids.

In [4]:
datalength = len(cmm)
print("We have",datalength,"records")

We have 1335576 records


In [5]:
if len(cmm['dim_claim_id'].unique())==datalength:
    print("There are",len(cmm['dim_claim_id'].unique()),"unique records, the same as the total number of records.")

There are 1335576 unique records, the same as the total number of records.


Next, we look at how many unique Payer BINs there are, and how many drugs. We see there are 4 payers represented, and 3 drugs.

In [6]:
print("There are",len(cmm['bin'].unique()),"unique payer BINs in the dataset.")

There are 4 unique payer BINs in the dataset.


In [7]:
print("There are",len(cmm['drug'].unique()),"unique drugs in the dataset.")

There are 3 unique drugs in the dataset.


Now, we check to make sure the only NaNs for PA info are for those claims where a PA form was not needed (and the claim was approved by pharmacy). This check is on reject_code, pa_approved, correct_diagnosis, contraindication, and tried_and_failed.

In [8]:
print("The number of claims with a PA form is",len(cmm[cmm['dim_pa_id'].notna()]),"and the number of claims",
      "that were rejected is",len(cmm[cmm['pharmacy_claim_approved']==0]),".")
if len(cmm[cmm['dim_pa_id'].notna()])==len(cmm[cmm['pharmacy_claim_approved']==0]):
    print("These are the same, so we can continue.")  
    pa_count=len(cmm[cmm['dim_pa_id'].notna()])
else:
    print("These differ, look back at claims that have claim not approved (pharmacy_claim_approved=0) but no corresponding PA id.")

The number of claims with a PA form is 555951 and the number of claims that were rejected is 555951 .
These are the same, so we can continue.


In [9]:
if np.sum(cmm[cmm['dim_pa_id'].notna()]['reject_code'].notna())==pa_count:
    print("There are",np.sum(cmm[cmm['dim_pa_id'].notna()]['reject_code'].notna()),"claims with a PA form and a reject_code, so no reject codes are missing.")

There are 555951 claims with a PA form and a reject_code, so no reject codes are missing.


In [10]:
if np.sum(cmm[cmm['dim_pa_id'].notna()]['pa_approved'].notna())==pa_count:
    print("There are",np.sum(cmm[cmm['dim_pa_id'].notna()]['pa_approved'].notna()),"claims with a PA form and an approval flag, so no results of the form (approved/denied) are missing.")

There are 555951 claims with a PA form and an approval flag, so no results of the form (approved/denied) are missing.


In [11]:
if np.sum(cmm[cmm['dim_pa_id'].notna()]['correct_diagnosis'].notna())==pa_count:
    print("There are",np.sum(cmm[cmm['dim_pa_id'].notna()]['correct_diagnosis'].notna()),"claims with a PA form and a correct_diagnosis flag, so no information on correct diagnosis are missing.")

There are 555951 claims with a PA form and a correct_diagnosis flag, so no information on correct diagnosis are missing.


In [12]:
if np.sum(cmm[cmm['dim_pa_id'].notna()]['tried_and_failed'].notna())==pa_count:
    print("There are",np.sum(cmm[cmm['dim_pa_id'].notna()]['tried_and_failed'].notna()),"claims with a PA form and a tried_and_failed flag, so no information on if patients tried and failed the generic alternatives is missing.")

There are 555951 claims with a PA form and a tried_and_failed flag, so no information on if patients tried and failed the generic alternatives is missing.


In [13]:
if np.sum(cmm[cmm['dim_pa_id'].notna()]['contraindication'].notna())==pa_count:
    print("There are",np.sum(cmm[cmm['dim_pa_id'].notna()]['contraindication'].notna()),"claims with a PA form and a contraindication flag, so no information on if patients have a contraindication to the requested drug is missing.")

There are 555951 claims with a PA form and a contraindication flag, so no information on if patients have a contraindication to the requested drug is missing.


Let us check if we can find any records where having same bin, drug, reject code, correct_diagnosis, tried_and_failed, contradictions, but have differnt pa_approved.

We will need to compare 4\*3\*3\*2\*2\*2 = 288 different groups of datas for different combination of bin, drug, reject code, correct_diagnosis, tried_and_failed, and contradictions. We will save these 288 groups as .csv files under folder \Data\tempData. They are named after 'bin_rc_cd_taf_ct_drug.csv', where rc stands for rejct_code, cd statnds for correct_diagnosis, taf stands for tried_and_failed, ct stands for contraindication.  

Since for now we are not taking date informations in to consideration, we can drop the corresponding columns to save some space. We would need to add dummy columns for drug type as for some reason query() does not like string comparisons.



In [14]:
cmm_cat = pd.concat([cmm, pd.get_dummies(cmm['drug']).copy()], axis=1) 
list(cmm_cat.columns)
cmm_cat = cmm_cat.drop(columns=['drug', 'date_val',
 'calendar_year',
 'calendar_month',
 'calendar_day',
 'day_of_week',
 'is_weekday',
 'is_workday',
 'is_holiday',])

We are only interested in the ones that were not approved by the pharmacy, so we only consider the ones that have proper reject codes.

In [15]:
cmm_bin = cmm.bin.unique()
cmm_drug = cmm.drug.unique()
cmm_reject_code = [75,76,70]
cmm_correct_diagnosis = [1,0]
cmm_tried_and_failed = [1,0]
cmm_contraindication = [1,0]

We used query() to select desired rows from the datafram and saved different groups as .csv files. 

The variable 'name' is a list which contains strings "bin_rc_cd_taf_ct_drug.csv".The variable 'havcont' is a list which index mates 'name'. havcont[i] = 1 if there are no contradicting data, 2 if there are contradicting data, 0 if there are no record that matches the combination of secelction. 

In [16]:
names = []
havecont = []
for bi in cmm_bin:
    for rc in cmm_reject_code:
        for cd in cmm_correct_diagnosis:
            for taf in cmm_tried_and_failed:
                for ct in cmm_contraindication:
                    nameA = str(bi)+'_'+str(rc)+'_'+str(cd)+'_'+str(taf)+'_'+str(cd)+'_'+str(ct)+'_A.csv'
                    name.append(nameA)
                    tempA = cmm_cat.query('bin == '+str(bi)+' and reject_code == '+str(rc)+ 
                                 ' and correct_diagnosis =='+str(cd)+' and tried_and_failed == '+str(taf)
                                 +' and contraindication =='+ str(ct)+'and A == 1')
                    tempA.to_csv('Data/tempData/'+nameA)
                    havecont.append(len(tempA.pa_approved.unique()))
                    nameB = str(bi)+'_'+str(rc)+'_'+str(cd)+'_'+str(taf)+'_'+str(cd)+'_'+str(ct)+'_B.csv'
                    name.append(nameB)
                    tempB = cmm_cat.query('bin == '+str(bi)+' and reject_code == '+str(rc)+ 
                                 ' and correct_diagnosis =='+str(cd)+' and tried_and_failed == '+str(taf)
                                 +' and contraindication =='+ str(ct)+'and B == 1')
                    tempB.to_csv('Data/tempData/'+nameB)
                    havecont.append(len(tempB.pa_approved.unique()))
                    nameC = str(bi)+'_'+str(rc)+'_'+str(cd)+'_'+str(taf)+'_'+str(cd)+'_'+str(ct)+'_C.csv'
                    name.append(nameC)
                    tempC = cmm_cat.query('bin == '+str(bi)+' and reject_code == '+str(rc)+ 
                                 ' and correct_diagnosis =='+str(cd)+' and tried_and_failed == '+str(taf)
                                 +' and contraindication == '+ str(ct)+' and C == 1')
                    tempC.to_csv('Data/tempData/'+nameC)
                    havecont.append(len(tempC.pa_approved.unique()))
                    
            


In [17]:
print('Among the 288 different combinations, there are '+
      str(havecont.count(0))+' of them have no matched records and '+ 
      str(havecont.count(2))+' of them have \'contradicting\' records. '+
      'There are none of the have \'not-contradicting\' records.')

Among the 288 different combinations, there are 192 of them have no matched records and 96 of them have 'contradicting' records. There are none of the have 'not-contradicting' records.


We can have a look of some recoreds that have 'contradicting' records.
Say, some of the data from 417380_75_1_1_1_1_A.csv (the group of records where bin = 417380, reject_code = 75, correct_diagnosis = 1, tried_and_failed = 1, contraindication = 1). 
The column 'Unnamed: 0	' is the previous index of the row from cmm.csv . 

In [18]:
cmm_417380_75_1_1_1_1_A = pd.read_csv("Data/tempData/417380_75_1_1_1_1_A.csv")

In [19]:
cmm_417380_75_1_1_1_1_A_approved = cmm_417380_75_1_1_1_1_A.loc[cmm_417380_75_1_1_1_1_A['pa_approved'] == 1]
cmm_417380_75_1_1_1_1_A_not_approved = cmm_417380_75_1_1_1_1_A.loc[cmm_417380_75_1_1_1_1_A['pa_approved'] == 0]

In [20]:
cmm_417380_75_1_1_1_1_A_approved.head()

,Unnamed: 0,dim_claim_id,dim_pa_id,dim_date_id,bin,reject_code,pharmacy_claim_approved,correct_diagnosis,tried_and_failed,contraindication,pa_approved,A,B,C
0,77,78,33.0,1,417380,75.0,0,1.0,1.0,1.0,1.0,1,0,0
1,310,311,131.0,2,417380,75.0,0,1.0,1.0,1.0,1.0,1,0,0
2,364,365,154.0,2,417380,75.0,0,1.0,1.0,1.0,1.0,1,0,0
3,410,411,175.0,2,417380,75.0,0,1.0,1.0,1.0,1.0,1,0,0
4,494,495,200.0,2,417380,75.0,0,1.0,1.0,1.0,1.0,1,0,0


In [21]:
cmm_417380_75_1_1_1_1_A_not_approved.head()

,Unnamed: 0,dim_claim_id,dim_pa_id,dim_date_id,bin,reject_code,pharmacy_claim_approved,correct_diagnosis,tried_and_failed,contraindication,pa_approved,A,B,C
9,1544,1545,638.0,3,417380,75.0,0,1.0,1.0,1.0,0.0,1,0,0
16,1834,1835,766.0,3,417380,75.0,0,1.0,1.0,1.0,0.0,1,0,0
145,30273,30274,12562.0,29,417380,75.0,0,1.0,1.0,1.0,0.0,1,0,0
161,33676,33677,13996.0,32,417380,75.0,0,1.0,1.0,1.0,0.0,1,0,0
194,42452,42453,17563.0,40,417380,75.0,0,1.0,1.0,1.0,0.0,1,0,0
